In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
#import keras

%matplotlib inline 

In [2]:
#Wenjie's 
ds_test = pd.read_csv("we_data/test.csv")
ds_train = pd.read_csv("we_data/train.csv")
ds_val = pd.read_csv("we_data/validation.csv")

In [3]:
df = ds_train

In [5]:
df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,250,FourthView,Na,0,10722,294,23,NaN,2821,NaN
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,90,OtherView,Na,133,7330,277,133,NaN,2259,NaN


In [10]:
# Average clickthrough rate
print(f"Base CTR: {df['click'].mean() * 100}%")

Base CTR: 0.07375623256619447%


In [ ]:
df['os', '']

In [21]:
df.groupby('useragent').mean()[['click']].sort_values('click', ascending=False)


,click
useragent,
ios_other,0.025641
android_other,0.006590
ios_safari,0.005698
android_safari,0.004839
android_chrome,0.004167
linux_chrome,0.004069
mac_safari,0.003235
mac_other,0.003083
mac_chrome,0.001546


In [118]:
x = df['useragent'].str.split('_', n=1, expand=True)
df['os'] = x[0]
df['browser'] = x[1]

In [24]:
df.groupby('advertiser').mean()[['click']]

,click
advertiser,
1458,0.000782
2259,0.000322
2261,0.000327
2821,0.000620
2997,0.004355
3358,0.000762
3386,0.000703
3427,0.000675
3476,0.000602


In [29]:
df.groupby('weekday').mean()[['click']]

,click
weekday,
0,0.000644
1,0.000676
2,0.000965
3,0.000833
4,0.000810
5,0.000725
6,0.000569


In [26]:
df.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [64]:
# Selected features
selected = ['useragent', 'advertiser', 'click', 'weekday', 'slotprice']

sel_data = df[selected]
sel_data.head()

,useragent,advertiser,click,weekday,slotprice
0,windows_ie,3427,0,5,5
1,windows_chrome,2821,0,1,0
2,windows_ie,3427,0,3,5
3,windows_ie,1458,0,6,0
4,windows_ie,2259,0,5,133


In [130]:
def preprocess(sel_data):
    selected = ['useragent', 'advertiser', 'click', 'weekday', 'slotprice']
    sel_data = sel_data[selected]
    # Slot price
    bins = [-np.inf, 10, 50, 100, np.inf]
    labels = ['sp_1','sp_2','sp_3','sp_4']
    sel_data_slot_bins = pd.cut(sel_data['slotprice'], bins=bins, labels=labels)
    sel_data_sb = pd.get_dummies(sel_data_slot_bins)

    # User Agent
    x = sel_data['useragent'].str.split('_', n=1, expand=True)
    sel_data['os'] = x[0]
    sel_data_os = pd.get_dummies(sel_data['os'])
    # Weekday
    sel_data_wd = pd.get_dummies(sel_data['weekday'])
    # Advertiser
    sel_data_ad = pd.get_dummies(sel_data['advertiser'])

    # Concat encoded data
    df = pd.concat([sel_data, sel_data_os, sel_data_wd, sel_data_ad, sel_data_sb], axis=1)

    # Drop the non essential
    df.drop(['os', 'useragent', 'advertiser', 'weekday', 'slotprice'], axis=1, inplace=True)
    
    return df

In [131]:
# Train Data
train_df = preprocess(ds_train)
X_train = train_df.drop('click', axis=1)
y_train = train_df['click']
val_df = preprocess(ds_val)
X_test = val_df.drop('click', axis=1)
y_test = val_df['click']

X_train.head()

/home/boonwenjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,android,ios,linux,mac,other,windows,0,1,2,3,...,2821,2997,3358,3386,3427,3476,sp_1,sp_2,sp_3,sp_4
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [132]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [133]:
prediction = logmodel.predict(X_test)

print('score:', logmodel.score(X_test, y_test))

from sklearn.metrics import classification_report

print(classification_report(y_test, prediction))

score: 0.9993353623426833
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    303723
           1       0.00      0.00      0.00       202

   micro avg       1.00      1.00      1.00    303925
   macro avg       0.50      0.50      0.50    303925
weighted avg       1.00      1.00      1.00    303925



/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [137]:
from sklearn.linear_model import SGDClassifier
sgdmodel = SGDClassifier(loss='log', max_iter=1000, tol=1e-3)
sgdmodel.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [138]:
prediction = sgdmodel.predict(X_test)
print('score:', logmodel.score(X_test, y_test))

print(classification_report(y_test, prediction))

score: 0.9993353623426833
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    303723
           1       0.00      0.00      0.00       202

   micro avg       1.00      1.00      1.00    303925
   macro avg       0.50      0.50      0.50    303925
weighted avg       1.00      1.00      1.00    303925



/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/boonwenjie/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
